# Análisis SQL y Exploratorio - Detección de Fraude Bancario
===========================================================

**Autor**: Ing. Daniel Varela Perez  
**Email**: bedaniele0@gmail.com  
**Teléfono**: +52 55 4189 3428  
**Fecha**: 24 de Septiembre, 2025  

**Objetivo**: Análisis profundo usando SQL para identificar patrones de fraude en transacciones bancarias

---

## 📊 Contenido
1. [Setup y Carga de Datos](#setup)
2. [Análisis SQL Básico](#sql-basic)
3. [Feature Engineering con SQL](#feature-engineering)
4. [Análisis de Desbalanceo](#imbalance)
5. [Visualizaciones](#visualizations)
6. [Insights y Conclusiones](#conclusions)

## 1. Setup y Carga de Datos

Configuramos el entorno e importamos las librerías necesarias para el análisis SQL.

In [ ]:
# Configuración inicial
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
import sys
from pathlib import Path

# Configuraciones
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Información del proyecto
print("🔍 ANÁLISIS SQL - DETECCIÓN DE FRAUDE BANCARIO")
print("=" * 60)
print(f"📊 Desarrollado por: Ing. Daniel Varela Perez")
print(f"📧 Email: bedaniele0@gmail.com")
print(f"📱 Tel: +52 55 4189 3428")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 60)

In [ ]:
# Cargar configuración del proyecto
sys.path.append('../src/utils')
from config import get_config
from environment import get_environment

config = get_config()
env = get_environment()

print("📋 CONFIGURACIÓN DEL PROYECTO:")
print(f"• Nombre: {config.get('project.name')}")
print(f"• Autor: {config.get('project.author')}")
print(f"• Entorno: {env.environment}")
print(f"• Dataset: {config.get('data.raw_path')}")

In [ ]:
# Cargar dataset de fraudes
print("📊 CARGANDO DATASET...")

# Ruta del dataset
dataset_path = '../data/raw/creditcard.csv'

# Cargar datos
df = pd.read_csv(dataset_path)

print(f"✅ Dataset cargado exitosamente")
print(f"📏 Dimensiones: {df.shape}")
print(f"💾 Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Información básica
print("\n📋 INFORMACIÓN BÁSICA:")
print(df.info())

In [ ]:
# Crear conexión SQLite en memoria
print("🗃️ CONFIGURANDO BASE DE DATOS SQLite...")

# Conexión en memoria para análisis rápido
conn = sqlite3.connect(':memory:')

# Cargar datos en tabla SQL
table_name = 'transactions'
df.to_sql(table_name, conn, index=False, if_exists='replace')

print(f"✅ Tabla '{table_name}' creada en SQLite")
print(f"📊 Registros cargados: {len(df):,}")

# Verificar estructura de la tabla
schema_query = f"PRAGMA table_info({table_name})"
schema_df = pd.read_sql_query(schema_query, conn)
print(f"\n🏗️ ESQUEMA DE LA TABLA:")
print(schema_df[['name', 'type']].head(10))

## 2. Análisis SQL Básico

Realizamos consultas SQL fundamentales para entender la distribución y características del dataset.

In [ ]:
# Query 1: Distribución de fraudes vs transacciones normales
fraud_dist_query = """
SELECT 
    Class,
    COUNT(*) as total_transactions,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM transactions), 4) as percentage,
    CASE 
        WHEN Class = 0 THEN 'Normal'
        WHEN Class = 1 THEN 'Fraud'
    END as transaction_type
FROM transactions
GROUP BY Class
ORDER BY Class;
"""

fraud_distribution = pd.read_sql_query(fraud_dist_query, conn)
print("📊 DISTRIBUCIÓN DE FRAUDES:")
print(fraud_distribution)

# Guardar resultado
fraud_rate = fraud_distribution[fraud_distribution['Class'] == 1]['percentage'].iloc[0]
print(f"\n🎯 TASA DE FRAUDE: {fraud_rate}%")
print(f"🚨 TOTAL DE FRAUDES: {fraud_distribution[fraud_distribution['Class'] == 1]['total_transactions'].iloc[0]:,}")

In [ ]:
# Query 2: Estadísticas por monto de transacción (SQLite compatible)
amount_stats_query = """
SELECT 
    Class,
    CASE 
        WHEN Class = 0 THEN 'Normal'
        WHEN Class = 1 THEN 'Fraud'
    END as transaction_type,
    COUNT(*) as count,
    ROUND(AVG(Amount), 2) as avg_amount,
    ROUND(MIN(Amount), 2) as min_amount,
    ROUND(MAX(Amount), 2) as max_amount
FROM transactions
GROUP BY Class
ORDER BY Class;
"""

amount_stats = pd.read_sql_query(amount_stats_query, conn)
print("💰 ESTADÍSTICAS DE MONTOS POR TIPO:")
print(amount_stats)

# Calcular percentiles usando pandas para complementar
normal_amounts = df[df['Class'] == 0]['Amount']
fraud_amounts = df[df['Class'] == 1]['Amount']

print("\n📊 PERCENTILES ADICIONALES:")
print(f"Normal - Q25: ${normal_amounts.quantile(0.25):.2f}, Mediana: ${normal_amounts.median():.2f}, Q75: ${normal_amounts.quantile(0.75):.2f}")
print(f"Fraude - Q25: ${fraud_amounts.quantile(0.25):.2f}, Mediana: ${fraud_amounts.median():.2f}, Q75: ${fraud_amounts.quantile(0.75):.2f}")

In [ ]:
# Query 3: Análisis temporal (variable Time)
temporal_analysis_query = """
WITH time_stats AS (
    SELECT 
        Class,
        Time,
        Amount,
        -- Convertir Time a horas (Time está en segundos desde primer transacción)
        ROUND(Time / 3600.0, 2) as hour_from_start,
        -- Crear buckets de tiempo para análisis
        CASE 
            WHEN Time < 86400 THEN 'Day_1'
            WHEN Time < 172800 THEN 'Day_2' 
            ELSE 'Later'
        END as time_bucket
    FROM transactions
)
SELECT 
    time_bucket,
    Class,
    CASE WHEN Class = 0 THEN 'Normal' ELSE 'Fraud' END as type,
    COUNT(*) as transaction_count,
    ROUND(AVG(Amount), 2) as avg_amount,
    ROUND(AVG(hour_from_start), 2) as avg_hour
FROM time_stats
GROUP BY time_bucket, Class
ORDER BY time_bucket, Class;
"""

temporal_analysis = pd.read_sql_query(temporal_analysis_query, conn)
print("⏰ ANÁLISIS TEMPORAL:")
print(temporal_analysis)

In [ ]:
# Query 4: Top 10 transacciones por monto (normales y fraudulentas)
top_transactions_query = """
SELECT type, Amount, Time, Class FROM (
    SELECT 'Normal' as type, Amount, Time, Class
    FROM transactions 
    WHERE Class = 0 
    ORDER BY Amount DESC 
    LIMIT 10

    UNION ALL

    SELECT 'Fraud' as type, Amount, Time, Class
    FROM transactions 
    WHERE Class = 1 
    ORDER BY Amount DESC 
    LIMIT 10
) ORDER BY type, Amount DESC;
"""

top_transactions = pd.read_sql_query(top_transactions_query, conn)
print("🔝 TOP TRANSACCIONES POR MONTO:")
print(top_transactions)

## 3. Feature Engineering con SQL

Creamos nuevas características usando window functions y análisis temporal avanzado.

In [ ]:
# Feature Engineering 1: Velocidad de transacciones
# Contar transacciones en ventanas de tiempo
velocity_query = """
WITH transaction_velocity AS (
    SELECT 
        *,
        -- Transacciones en la última hora (3600 segundos)
        COUNT(*) OVER (
            ORDER BY Time 
            RANGE BETWEEN 3600 PRECEDING AND CURRENT ROW
        ) as transactions_last_hour,
        
        -- Transacciones en las últimas 6 horas 
        COUNT(*) OVER (
            ORDER BY Time 
            RANGE BETWEEN 21600 PRECEDING AND CURRENT ROW
        ) as transactions_last_6hours,
        
        -- Tiempo desde la transacción anterior
        Time - LAG(Time, 1, Time) OVER (ORDER BY Time) as time_since_last
    FROM transactions
    ORDER BY Time
)
SELECT 
    Class,
    CASE WHEN Class = 0 THEN 'Normal' ELSE 'Fraud' END as type,
    ROUND(AVG(transactions_last_hour), 2) as avg_velocity_1h,
    ROUND(AVG(transactions_last_6hours), 2) as avg_velocity_6h,
    ROUND(AVG(time_since_last), 2) as avg_time_between,
    COUNT(*) as total_transactions
FROM transaction_velocity
GROUP BY Class
ORDER BY Class;
"""

velocity_analysis = pd.read_sql_query(velocity_query, conn)
print("⚡ ANÁLISIS DE VELOCIDAD DE TRANSACCIONES:")
print(velocity_analysis)

In [ ]:
# Feature Engineering 2: Frecuencia por rangos de monto
amount_frequency_query = """
WITH amount_ranges AS (
    SELECT 
        *,
        CASE 
            WHEN Amount = 0 THEN 'Zero'
            WHEN Amount > 0 AND Amount <= 10 THEN 'Very_Low (0-10)'
            WHEN Amount > 10 AND Amount <= 50 THEN 'Low (10-50)'
            WHEN Amount > 50 AND Amount <= 100 THEN 'Medium (50-100)'
            WHEN Amount > 100 AND Amount <= 500 THEN 'High (100-500)'
            WHEN Amount > 500 AND Amount <= 1000 THEN 'Very_High (500-1000)'
            ELSE 'Extreme (>1000)'
        END as amount_range
    FROM transactions
)
SELECT 
    amount_range,
    Class,
    COUNT(*) as frequency,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Class), 2) as percentage_within_class,
    ROUND(AVG(Amount), 2) as avg_amount_in_range,
    ROUND(SUM(CASE WHEN Class = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as fraud_rate_in_range
FROM amount_ranges
GROUP BY amount_range, Class
ORDER BY 
    CASE amount_range
        WHEN 'Zero' THEN 1
        WHEN 'Very_Low (0-10)' THEN 2
        WHEN 'Low (10-50)' THEN 3
        WHEN 'Medium (50-100)' THEN 4
        WHEN 'High (100-500)' THEN 5
        WHEN 'Very_High (500-1000)' THEN 6
        WHEN 'Extreme (>1000)' THEN 7
    END, Class;
"""

amount_frequency = pd.read_sql_query(amount_frequency_query, conn)
print("💵 ANÁLISIS DE FRECUENCIA POR RANGOS DE MONTO:")
print(amount_frequency)

In [ ]:
# Feature Engineering 3: Detección de patrones anómalos
anomaly_patterns_query = """
WITH anomaly_features AS (
    SELECT 
        *,
        -- Z-score para Amount
        ROUND((Amount - AVG(Amount) OVER ()) / NULLIF(STDDEV(Amount) OVER (), 0), 3) as amount_zscore,
        
        -- Ranking por monto
        NTILE(100) OVER (ORDER BY Amount) as amount_percentile,
        
        -- Diferencia con monto promedio en ventana de tiempo
        ROUND(Amount - AVG(Amount) OVER (
            ORDER BY Time 
            RANGE BETWEEN 3600 PRECEDING AND 3600 FOLLOWING
        ), 2) as amount_diff_local_avg,
        
        -- Transacciones consecutivas rápidas (< 60 segundos)
        CASE 
            WHEN Time - LAG(Time, 1, 0) OVER (ORDER BY Time) < 60 THEN 1
            ELSE 0
        END as is_rapid_transaction
        
    FROM transactions
)
SELECT 
    Class,
    CASE WHEN Class = 0 THEN 'Normal' ELSE 'Fraud' END as type,
    
    -- Estadísticas de Z-score
    ROUND(AVG(ABS(amount_zscore)), 3) as avg_abs_zscore,
    ROUND(AVG(amount_zscore), 3) as avg_zscore,
    
    -- Transacciones en percentiles extremos (>95 o <5)
    ROUND(SUM(CASE WHEN amount_percentile > 95 OR amount_percentile < 5 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as pct_extreme_amounts,
    
    -- Diferencia promedio con contexto local
    ROUND(AVG(ABS(amount_diff_local_avg)), 2) as avg_local_deviation,
    
    -- Porcentaje de transacciones rápidas
    ROUND(AVG(is_rapid_transaction) * 100, 2) as pct_rapid_transactions,
    
    COUNT(*) as total_transactions
    
FROM anomaly_features
GROUP BY Class
ORDER BY Class;
"""

anomaly_patterns = pd.read_sql_query(anomaly_patterns_query, conn)
print("🔍 DETECCIÓN DE PATRONES ANÓMALOS:")
print(anomaly_patterns)

## 4. Análisis de Desbalanceo

Analizamos en profundidad el desbalanceo del dataset y definimos estrategias.

In [ ]:
# Análisis detallado del desbalanceo
imbalance_query = """
WITH class_stats AS (
    SELECT 
        Class,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM transactions), 6) as percentage
    FROM transactions
    GROUP BY Class
),
ratios AS (
    SELECT 
        (SELECT count FROM class_stats WHERE Class = 0) as normal_count,
        (SELECT count FROM class_stats WHERE Class = 1) as fraud_count,
        ROUND(
            (SELECT count FROM class_stats WHERE Class = 0) * 1.0 / 
            (SELECT count FROM class_stats WHERE Class = 1), 2
        ) as imbalance_ratio
)
SELECT 
    cs.*,
    CASE WHEN Class = 0 THEN 'Normal' ELSE 'Fraud' END as type,
    r.imbalance_ratio,
    -- Calcular cuántos ejemplos necesitaríamos para diferentes estrategias
    CASE 
        WHEN Class = 1 THEN 
            ROUND(r.normal_count * 0.1) - count  -- Para ratio 10:1
        ELSE NULL
    END as samples_needed_10_to_1,
    CASE 
        WHEN Class = 1 THEN 
            ROUND(r.normal_count * 0.05) - count  -- Para ratio 20:1
        ELSE NULL
    END as samples_needed_20_to_1
FROM class_stats cs
CROSS JOIN ratios r
ORDER BY Class;
"""

imbalance_analysis = pd.read_sql_query(imbalance_query, conn)
print("⚖️ ANÁLISIS DETALLADO DE DESBALANCEO:")
print(imbalance_analysis)

# Extraer información clave
imbalance_ratio = imbalance_analysis['imbalance_ratio'].iloc[0]
fraud_count = imbalance_analysis[imbalance_analysis['Class'] == 1]['count'].iloc[0]
normal_count = imbalance_analysis[imbalance_analysis['Class'] == 0]['count'].iloc[0]

print(f"\n📊 MÉTRICAS CLAVE:")
print(f"• Ratio de desbalanceo: {imbalance_ratio}:1 (Normal:Fraud)")
print(f"• Transacciones normales: {normal_count:,}")
print(f"• Transacciones fraudulentas: {fraud_count:,}")
print(f"• Tasa de fraude: {fraud_count/(fraud_count + normal_count)*100:.4f}%")

In [ ]:
# Análisis por percentiles usando pandas (SQLite no soporta PERCENTILE)
print("📊 ANÁLISIS POR PERCENTILES:")

# Función para calcular percentiles por clase
def calculate_percentiles(data, class_label):
    amounts = data[data['Class'] == class_label]['Amount']
    return {
        'Class': class_label,
        'type': 'Normal' if class_label == 0 else 'Fraud',
        'count': len(amounts),
        'min_amount': amounts.min(),
        'p01_amount': amounts.quantile(0.01),
        'p05_amount': amounts.quantile(0.05),
        'p10_amount': amounts.quantile(0.10),
        'p25_amount': amounts.quantile(0.25),
        'p50_amount': amounts.quantile(0.50),
        'p75_amount': amounts.quantile(0.75),
        'p90_amount': amounts.quantile(0.90),
        'p95_amount': amounts.quantile(0.95),
        'p99_amount': amounts.quantile(0.99),
        'max_amount': amounts.max(),
        'mean_amount': amounts.mean(),
        'std_amount': amounts.std()
    }

# Calcular percentiles para ambas clases
percentiles_normal = calculate_percentiles(df, 0)
percentiles_fraud = calculate_percentiles(df, 1)

# Convertir a DataFrame para mejor visualización
percentile_analysis = pd.DataFrame([percentiles_normal, percentiles_fraud])

# Mostrar análisis transpuesto
print(percentile_analysis.round(2).T)

# Comparar diferencias clave
print("\n🎯 COMPARACIÓN CLAVE:")
print(f"• Mediana Normal: ${percentiles_normal['p50_amount']:.2f}")
print(f"• Mediana Fraude: ${percentiles_fraud['p50_amount']:.2f}")
print(f"• Media Normal: ${percentiles_normal['mean_amount']:.2f}")
print(f"• Media Fraude: ${percentiles_fraud['mean_amount']:.2f}")
print(f"• P95 Normal: ${percentiles_normal['p95_amount']:.2f}")
print(f"• P95 Fraude: ${percentiles_fraud['p95_amount']:.2f}")

## 5. Visualizaciones

Creamos visualizaciones para identificar patrones visuales en los datos.

In [ ]:
# Visualización 1: Distribución de montos (Normal vs Fraud)
print("📊 Creando visualización de distribución de montos...")

# Preparar datos
normal_amounts = df[df['Class'] == 0]['Amount']
fraud_amounts = df[df['Class'] == 1]['Amount']

# Crear subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análisis de Distribución de Montos - Normal vs Fraude\nDesarrollado por: Ing. Daniel Varela Perez', 
             fontsize=16, fontweight='bold')

# Histograma comparativo
axes[0,0].hist(normal_amounts, bins=50, alpha=0.7, label='Normal', color='blue', density=True)
axes[0,0].hist(fraud_amounts, bins=50, alpha=0.7, label='Fraude', color='red', density=True)
axes[0,0].set_xlabel('Monto ($)')
axes[0,0].set_ylabel('Densidad')
axes[0,0].set_title('Distribución de Montos (Escala Normal)')
axes[0,0].legend()

# Histograma en escala log
axes[0,1].hist(normal_amounts[normal_amounts > 0], bins=50, alpha=0.7, label='Normal', color='blue', density=True)
axes[0,1].hist(fraud_amounts[fraud_amounts > 0], bins=50, alpha=0.7, label='Fraude', color='red', density=True)
axes[0,1].set_xscale('log')
axes[0,1].set_xlabel('Monto ($) - Escala Log')
axes[0,1].set_ylabel('Densidad')
axes[0,1].set_title('Distribución de Montos (Escala Logarítmica)')
axes[0,1].legend()

# Box plot comparativo
box_data = [normal_amounts, fraud_amounts]
box_labels = ['Normal', 'Fraude']
bp = axes[1,0].boxplot(box_data, labels=box_labels, patch_artist=True)
bp['boxes'][0].set_facecolor('blue')
bp['boxes'][1].set_facecolor('red')
axes[1,0].set_ylabel('Monto ($)')
axes[1,0].set_title('Box Plot de Montos por Clase')

# Violin plot
violin_data = pd.DataFrame({
    'Amount': np.concatenate([normal_amounts.values[:5000], fraud_amounts.values]),  # Sample para performance
    'Class': ['Normal']*5000 + ['Fraud']*len(fraud_amounts)
})
sns.violinplot(data=violin_data, x='Class', y='Amount', ax=axes[1,1])
axes[1,1].set_title('Violin Plot de Montos por Clase')

plt.tight_layout()
plt.savefig('../reports/figures/amount_distribution_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualización guardada en reports/figures/amount_distribution_analysis.png")

In [ ]:
# Visualización 2: Patrones temporales
print("⏰ Creando visualización de patrones temporales...")

# Preparar datos temporales
df_temp = df.copy()
df_temp['Hours'] = df_temp['Time'] / 3600  # Convertir a horas
df_temp['Days'] = df_temp['Hours'] / 24    # Convertir a días

# Crear visualización temporal
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análisis de Patrones Temporales\nDesarrollado por: Ing. Daniel Varela Perez', 
             fontsize=16, fontweight='bold')

# Distribución temporal de fraudes
normal_times = df_temp[df_temp['Class'] == 0]['Hours']
fraud_times = df_temp[df_temp['Class'] == 1]['Hours']

axes[0,0].hist(normal_times, bins=100, alpha=0.7, label='Normal', color='blue', density=True)
axes[0,0].hist(fraud_times, bins=100, alpha=0.7, label='Fraude', color='red', density=True)
axes[0,0].set_xlabel('Tiempo (Horas desde inicio)')
axes[0,0].set_ylabel('Densidad')
axes[0,0].set_title('Distribución Temporal de Transacciones')
axes[0,0].legend()

# Tasa de fraude por hora del día (simulada)
df_temp['Hour_of_Day'] = (df_temp['Hours'] % 24).astype(int)
hourly_fraud_rate = df_temp.groupby('Hour_of_Day').agg({
    'Class': ['count', 'sum']
}).reset_index()
hourly_fraud_rate.columns = ['Hour', 'Total', 'Frauds']
hourly_fraud_rate['Fraud_Rate'] = hourly_fraud_rate['Frauds'] / hourly_fraud_rate['Total'] * 100

axes[0,1].bar(hourly_fraud_rate['Hour'], hourly_fraud_rate['Fraud_Rate'], color='orange', alpha=0.7)
axes[0,1].set_xlabel('Hora del Día')
axes[0,1].set_ylabel('Tasa de Fraude (%)')
axes[0,1].set_title('Tasa de Fraude por Hora del Día')

# Evolución temporal de montos
# Agrupar por ventanas de tiempo
df_temp['Time_Window'] = (df_temp['Hours'] // 6).astype(int)  # Ventanas de 6 horas
time_amount_stats = df_temp.groupby(['Time_Window', 'Class'])['Amount'].mean().unstack(fill_value=0)

axes[1,0].plot(time_amount_stats.index, time_amount_stats[0], label='Normal', color='blue', linewidth=2)
axes[1,0].plot(time_amount_stats.index, time_amount_stats[1], label='Fraude', color='red', linewidth=2)
axes[1,0].set_xlabel('Ventana de Tiempo (6h)')
axes[1,0].set_ylabel('Monto Promedio ($)')
axes[1,0].set_title('Evolución Temporal de Montos Promedio')
axes[1,0].legend()

# Heatmap de fraudes por día y hora
df_temp['Day'] = (df_temp['Hours'] // 24).astype(int)
df_sample = df_temp[df_temp['Day'] < 7]  # Primeros 7 días
pivot_data = df_sample.groupby(['Day', 'Hour_of_Day'])['Class'].sum().unstack(fill_value=0)

sns.heatmap(pivot_data, cmap='Reds', ax=axes[1,1], cbar_kws={'label': 'Número de Fraudes'})
axes[1,1].set_xlabel('Hora del Día')
axes[1,1].set_ylabel('Día')
axes[1,1].set_title('Heatmap de Fraudes (Primeros 7 días)')

plt.tight_layout()
plt.savefig('../reports/figures/temporal_patterns_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualización guardada en reports/figures/temporal_patterns_analysis.png")

In [ ]:
# Visualización 3: Correlaciones entre variables
print("🔗 Creando análisis de correlaciones...")

# Seleccionar subset de variables para correlación (V1-V10 + Amount + Class)
correlation_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'Amount', 'Class']
corr_data = df[correlation_cols]

# Calcular matriz de correlación
correlation_matrix = corr_data.corr()

# Crear visualización
fig, axes = plt.subplots(1, 2, figsize=(20, 8))
fig.suptitle('Análisis de Correlaciones\nDesarrollado por: Ing. Daniel Varela Perez', 
             fontsize=16, fontweight='bold')

# Heatmap de correlaciones
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0, 
            square=True,
            fmt='.2f',
            ax=axes[0])
axes[0].set_title('Matriz de Correlación (Variables V1-V10 + Amount + Class)')

# Correlaciones específicas con la variable Class
class_correlations = correlation_matrix['Class'].drop('Class').sort_values(key=abs, ascending=False)

# Bar plot de correlaciones con Class
colors = ['red' if x < 0 else 'green' for x in class_correlations.values]
axes[1].bar(range(len(class_correlations)), class_correlations.values, color=colors, alpha=0.7)
axes[1].set_xticks(range(len(class_correlations)))
axes[1].set_xticklabels(class_correlations.index, rotation=45)
axes[1].set_ylabel('Correlación con Class')
axes[1].set_title('Correlaciones con Variable Objetivo (Class)')
axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/correlation_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Mostrar correlaciones más significativas
print("\n🎯 CORRELACIONES MÁS SIGNIFICATIVAS CON FRAUDE:")
for var, corr in class_correlations.head(10).items():
    print(f"• {var}: {corr:.4f}")

print("\n✅ Visualización guardada en reports/figures/correlation_analysis.png")

## 6. Insights y Conclusiones

Documentamos los hallazgos principales y patrones identificados.

In [ ]:
# Recopilar insights clave del análisis
print("🎯 INSIGHTS CLAVE IDENTIFICADOS")
print("=" * 60)

# Insight 1: Desbalanceo extremo
print(f"\n1. 📊 DESBALANCEO EXTREMO:")
print(f"   • Ratio: {imbalance_ratio:.1f}:1 (Normal:Fraude)")
print(f"   • Solo {fraud_rate:.4f}% de transacciones son fraude")
print(f"   • Requiere técnicas especializadas de balanceado")

# Insight 2: Diferencias en montos
normal_avg = amount_stats[amount_stats['Class'] == 0]['avg_amount'].iloc[0]
fraud_avg = amount_stats[amount_stats['Class'] == 1]['avg_amount'].iloc[0]
print(f"\n2. 💰 PATRONES DE MONTOS:")
print(f"   • Monto promedio normal: ${normal_avg:.2f}")
print(f"   • Monto promedio fraude: ${fraud_avg:.2f}")
print(f"   • Diferencia: {abs(normal_avg - fraud_avg):.2f} ({'menor' if fraud_avg < normal_avg else 'mayor'} en fraudes)")

# Insight 3: Patrones temporales
print(f"\n3. ⏰ DISTRIBUCIÓN TEMPORAL:")
print(f"   • Fraudes distribuidos a lo largo del tiempo")
print(f"   • No se observan concentraciones claras por hora")
print(f"   • Patrón temporal relativamente uniforme")

# Insight 4: Variables más correlacionadas
top_corr_vars = abs(class_correlations).head(3)
print(f"\n4. 🔗 VARIABLES MÁS PREDICTIVAS:")
for var, corr in top_corr_vars.items():
    direction = "positiva" if class_correlations[var] > 0 else "negativa"
    print(f"   • {var}: {corr:.4f} (correlación {direction})")

# Insight 5: Anomalías detectadas
normal_anomalies = anomaly_patterns[anomaly_patterns['Class'] == 0]
fraud_anomalies = anomaly_patterns[anomaly_patterns['Class'] == 1]
print(f"\n5. 🚨 DETECCIÓN DE ANOMALÍAS:")
print(f"   • Transacciones normales con montos extremos: {normal_anomalies['pct_extreme_amounts'].iloc[0]:.2f}%")
print(f"   • Transacciones fraude con montos extremos: {fraud_anomalies['pct_extreme_amounts'].iloc[0]:.2f}%")
print(f"   • Transacciones rápidas en fraudes: {fraud_anomalies['pct_rapid_transactions'].iloc[0]:.2f}%")

print(f"\n" + "=" * 60)
print(f"👨‍💻 Análisis realizado por: Ing. Daniel Varela Perez")
print(f"📧 Email: bedaniele0@gmail.com | 📱 Tel: +52 55 4189 3428")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"=" * 60)

In [ ]:
# Recomendaciones para el modelado
print("🚀 RECOMENDACIONES PARA MODELADO ML")
print("=" * 60)

print("\n1. 📊 ESTRATEGIAS DE BALANCEADO:")
print("   • Usar SMOTE o ADASYN para oversampling")
print("   • Considerar undersampling de mayoría")
print("   • Implementar ensemble con diferentes ratios")
print("   • Usar cost-sensitive learning")

print("\n2. 🎯 MÉTRICAS DE EVALUACIÓN:")
print("   • Priorizar Precision, Recall y F1-Score")
print("   • Usar AUC-ROC y AUC-PR")
print("   • Evitar Accuracy como métrica principal")
print("   • Implementar matrices de confusión detalladas")

print("\n3. 🔧 FEATURE ENGINEERING:")
print("   • Variables temporales (velocity features)")
print("   • Agregaciones por ventanas de tiempo")
print("   • Ratios entre variables existentes")
print("   • Interacciones entre variables V1-V28")

print("\n4. 🤖 ALGORITMOS RECOMENDADOS:")
print("   • XGBoost (excelente para desbalanceados)")
print("   • LightGBM (rápido y eficiente)")
print("   • Random Forest con class_weight='balanced'")
print("   • Ensemble de múltiples algoritmos")

print("\n5. ✅ VALIDACIÓN:")
print("   • Usar StratifiedKFold para mantener distribución")
print("   • Time-based split si hay orden temporal")
print("   • Validar en datos completamente separados")
print("   • Monitorear drift en producción")

print(f"\n" + "=" * 60)
print(f"🎯 OBJETIVO: Precision > 99.5% y Recall > 85%")
print(f"📊 Dataset: {len(df):,} transacciones, {fraud_count:,} fraudes")
print(f"⚡ Listo para Fase 2: Feature Engineering y Modelado")
print(f"=" * 60)

In [ ]:
# Limpiar conexiones y guardar resultados
print("🧹 LIMPIANDO RECURSOS...")

# Cerrar conexión SQLite
conn.close()
print("✅ Conexión SQLite cerrada")

# Crear resumen de resultados
summary_results = {
    'dataset_info': {
        'total_transactions': len(df),
        'fraud_transactions': fraud_count,
        'normal_transactions': normal_count,
        'fraud_rate_percent': fraud_rate,
        'imbalance_ratio': imbalance_ratio
    },
    'amount_statistics': {
        'normal_avg_amount': float(normal_avg),
        'fraud_avg_amount': float(fraud_avg)
    },
    'top_correlations': dict(class_correlations.head(5)),
    'analysis_date': datetime.now().isoformat(),
    'analyst': 'Ing. Daniel Varela Perez',
    'contact': 'bedaniele0@gmail.com'
}

# Guardar resultados
import json
with open('../reports/sql_eda_results.json', 'w') as f:
    json.dump(summary_results, f, indent=2)

print("✅ Resultados guardados en reports/sql_eda_results.json")
print("📊 Análisis SQL completado exitosamente")
print("🚀 Listo para continuar con Feature Engineering")